# Import Library

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

# Data Loading & Preprocessing

In [2]:
path = r"https://raw.githubusercontent.com/DR21-ds/Spam--Ham-Mail-Prediction/main/spamham.csv"
data = pd.read_csv(path, error_bad_lines=False)
mail_data = data.where((pd.notnull(data)), '')

In [3]:
mail_data.shape

(5572, 2)

In [4]:
mail_data.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
mail_data.loc[mail_data['Category'] == 'spam' , 'Category'] = 0
mail_data.loc[mail_data['Category'] == 'ham' , 'Category'] = 1

In [6]:
mail_data

,Category,Message
0,1,"Go until jurong point, crazy.. Available only ..."
1,1,Ok lar... Joking wif u oni...
2,0,Free entry in 2 a wkly comp to win FA Cup fina...
3,1,U dun say so early hor... U c already then say...
4,1,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,0,This is the 2nd time we have tried 2 contact u...
5568,1,Will ü b going to esplanade fr home?
5569,1,"Pity, * was in mood for that. So...any other s..."
5570,1,The guy did some bitching but I acted like i'd...


In [7]:
x=mail_data['Message']
y=mail_data['Category']

In [8]:
print(x)
print(y)

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                 Will ü b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: Message, Length: 5572, dtype: object
0       1
1       1
2       0
3       1
4       1
       ..
5567    0
5568    1
5569    1
5570    1
5571    1
Name: Category, Length: 5572, dtype: object


# Traininig and Testing 

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 0.8, test_size = 0.2, random_state = 3)

# Future Extraction

In [10]:
feature_extraction = TfidfVectorizer(min_df=1, stop_words='english', lowercase='True')
x_train_features = feature_extraction.fit_transform(x_train)
x_test_features = feature_extraction.transform(x_test)

y_train = y_train.astype('int')
y_test = y_test.astype('int')

# Support Vector Machine

In [11]:
model = LinearSVC()
model.fit(x_train_features, y_train)

LinearSVC()

## Evaluation of Model

In [12]:
prediction_on_training_data = model.predict(x_train_features)
accuracy_on_training_data = accuracy_score(y_train, prediction_on_training_data)

In [13]:
print('Accuracy on Training data : ' , round((accuracy_on_training_data)*100,2))

Accuracy on Training data :  99.93


In [14]:
prediction_on_test_data = model.predict(x_test_features)
accuracy_on_test_data = accuracy_score(y_test, prediction_on_test_data)
print('Accuracy on Test data : ' , round((accuracy_on_test_data)*100,2))

Accuracy on Test data :  98.21


# K Nearest Neighbors

In [15]:
from sklearn.neighbors import KNeighborsClassifier

In [16]:
k_model=KNeighborsClassifier()
k_model.fit(x_train_features,y_train)

KNeighborsClassifier()

## Evaluation of Model

In [17]:
prediction_on_training_data = k_model.predict(x_train_features)
accuracy_on_training_data = accuracy_score(y_train, prediction_on_training_data)
print('Accuracy on Training data : ' , round((accuracy_on_training_data)*100,2))
prediction_on_test_data = k_model.predict(x_test_features)
accuracy_on_test_data = accuracy_score(y_test, prediction_on_test_data)
print('Accuracy on Test data : ' , round((accuracy_on_test_data)*100,2))

Accuracy on Training data :  92.01
Accuracy on Test data :  90.94


## Evaluation of Model with hyperparameter Tuning

In [18]:
from sklearn.model_selection import RandomizedSearchCV

In [19]:
n_neighbors = [1,2,3,4,5,6,7,8,9,10,12]
weights=['uniform','distance']
leaf_size=[20,30,40,50,60,70]
p=[1,2]

hyperparameter_grid = {
    'n_neighbors': n_neighbors,
    'weights':weights,
    'leaf_size':leaf_size,
    'p':p
    }

random_cv = RandomizedSearchCV(estimator=k_model,
            param_distributions=hyperparameter_grid,
            cv=5, n_iter=50,
            n_jobs = 4,
            verbose = 5, 
            return_train_score = True,
            random_state=42)

In [20]:
random_cv.fit(x_train_features,y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:   11.6s
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:   44.1s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:  1.5min
[Parallel(n_jobs=4)]: Done 250 out of 250 | elapsed:  2.5min finished


RandomizedSearchCV(cv=5, estimator=KNeighborsClassifier(), n_iter=50, n_jobs=4,
                   param_distributions={'leaf_size': [20, 30, 40, 50, 60, 70],
                                        'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8,
                                                        9, 10, 12],
                                        'p': [1, 2],
                                        'weights': ['uniform', 'distance']},
                   random_state=42, return_train_score=True, verbose=5)

In [21]:
random_cv.best_estimator_

KNeighborsClassifier(n_neighbors=1, p=1, weights='distance')

In [22]:
k_model=KNeighborsClassifier(n_neighbors=1, p=1, weights='distance')
k_model.fit(x_train_features,y_train)
prediction_on_training_data = k_model.predict(x_train_features)
accuracy_on_training_data = accuracy_score(y_train, prediction_on_training_data)
print('Accuracy on Training data : ' , round((accuracy_on_training_data)*100,2))
prediction_on_test_data = k_model.predict(x_test_features)
accuracy_on_test_data = accuracy_score(y_test, prediction_on_test_data)
print('Accuracy on Test data : ' , round((accuracy_on_test_data)*100,2))

Accuracy on Training data :  100.0
Accuracy on Test data :  94.53


# Naive Bayes

In [23]:
from sklearn.naive_bayes import GaussianNB

In [24]:
n_model = GaussianNB()
n_model.fit(x_train_features.todense(),y_train)

GaussianNB()

## Evaluation of Model

In [25]:
prediction_on_training_data = n_model.predict(x_train_features.todense())
accuracy_on_training_data = accuracy_score(y_train, prediction_on_training_data)
print('Accuracy on Training data : ' , round((accuracy_on_training_data)*100,2))
prediction_on_test_data = n_model.predict(x_test_features.todense())
accuracy_on_test_data = accuracy_score(y_test, prediction_on_test_data)
print('Accuracy on Test data : ' , round((accuracy_on_test_data)*100,2))

Accuracy on Training data :  93.47
Accuracy on Test data :  88.79


# Final Selection
- From All the Model SVC Score is good among all so select svc and predict few example.

In [26]:
input_mail = ['07732584351 - Rodger Burns - MSG = We tried to call you re your reply to our sms for a free nokia mobile + free camcorder. Please call now 08000930705 for delivery tomorrow']

input_mail_features = feature_extraction.transform(input_mail)

# Making Prediction
prediction = model.predict(input_mail_features)
print(prediction)

if (prediction[0] == 1):
    print('Ham Mail')
else:
    print('Spam Mail')

[0]
Spam Mail


In [27]:
input_mail1 = ["Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."]

# convert text to feature vectors
input_mail_features1 = feature_extraction.transform(input_mail1)

# Making Prediction
prediction1 = model.predict(input_mail_features1)
print(prediction1)

if (prediction1[0] == 1):
    print('Ham Mail')
else:
    print('Spam Mail')

[1]
Ham Mail
